In [1]:
import os

# Disable WandB logging
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!pip install accelerate>=0.21.0
!pip install transformers[torch]
!pip install transformers datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 83.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

2024-07-13 12:35:43.823510: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 12:35:43.823622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 12:35:43.948874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
dataset = load_dataset('HooshvareLab/pn_summary', trust_remote_code=True)

Generating train split:   0%|          | 0/82022 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5592 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5593 [00:00<?, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("nafisehNik/mt5-persian-summary")

def preprocess_data(examples):
    inputs = examples['article']
    targets = examples['title']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_data, batched=True)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/82022 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5592 [00:00<?, ? examples/s]

Map:   0%|          | 0/5593 [00:00<?, ? examples/s]

In [6]:
tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.1)
train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['test']

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("nafisehNik/mt5-persian-summary")

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=None, 
    save_steps=1e10, 
    num_train_epochs=3,
    predict_with_generate=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.280000,0.251913
2,0.272300,0.246292
3,0.265600,0.244877


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 120, 'early_stopping': True, 'num_beams': 2, 'length_penalty': 2.0}


TrainOutput(global_step=55365, training_loss=0.6555912193864016, metrics={'train_runtime': 13690.0635, 'train_samples_per_second': 16.176, 'train_steps_per_second': 4.044, 'total_flos': 1.1709532029321216e+17, 'train_loss': 0.6555912193864016, 'epoch': 3.0})

In [8]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.24487707018852234, 'eval_runtime': 124.173, 'eval_samples_per_second': 66.061, 'eval_steps_per_second': 16.517, 'epoch': 3.0}


In [17]:
!pip install sacrebleu rouge-score bert-score

import torch
import sacrebleu
from rouge_score import rouge_scorer
from bert_score import score
from tqdm import tqdm
import shutil
from IPython.display import FileLink

# Ensure you are using the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Selected device: {device}")

# Generate headline function
def generate_headline(article_text):
    inputs = tokenizer(article_text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the GPU
    outputs = model.generate(**inputs)
    headline = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return headline

# Prepare for evaluation metrics
reference_texts = [example['title'] for example in val_dataset]
generated_texts = []

# Generate summaries for the validation dataset with progress bar
print("Generating summaries...")
for example in tqdm(val_dataset, desc="Processing"):
    article_text = example['article']
    headline = generate_headline(article_text)
    generated_texts.append(headline)

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(generated_texts, [reference_texts])
print(f"BLEU score: {bleu.score}")

# Calculate ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(ref, gen) for ref, gen in zip(reference_texts, generated_texts)]
rouge1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
rouge2 = sum(score['rouge2'].fmeasure for score in rouge_scores) / len(rouge_scores)
rougeL = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)
print(f"ROUGE-1: {rouge1}")
print(f"ROUGE-2: {rouge2}")
print(f"ROUGE-L: {rougeL}")

# Calculate BERTScore
print("Calculating BERTScore...")
P, R, F1 = score(generated_texts, reference_texts, lang="fa", device=device)
print(f"BERTScore Precision: {P.mean().item()}")
print(f"BERTScore Recall: {R.mean().item()}")
print(f"BERTScore F1: {F1.mean().item()}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Selected device: cuda
Generating summaries...


Processing: 100%|██████████| 8203/8203 [40:02<00:00,  3.41it/s]


BLEU score: 10.049115381380938
ROUGE-1: 0.005189158437969848
ROUGE-2: 0.00036571985858832134
ROUGE-L: 0.005189158437969848
Calculating BERTScore...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore Precision: 0.7613582611083984
BERTScore Recall: 0.7612707018852234
BERTScore F1: 0.7609006762504578


In [18]:
article_text = "به گزارش شانا به نقل از شرکت ملی صنایع پتروشیمی، علی‌اصغر گودرزی‌فراهانی با اشاره به اینکه همه طرح‌های در حال اجرای صنعت پتروشیمی براساس پیشرفت فیزیکی و پیش‌بینی زمان راه‌اندازی در قالب طرح‌های جهش دوم و سوم تقسیم‌بندی شده‌اند، اظهار کرد: انتظار داریم که طرح‌های جهش دوم صنعت پتروشیمی که پیشرفت‌های بسیار خوبی نیز دارند تا پایان سال ۱۴۰۰ و در واقع تا پایان برنامه ششم توسعه به نتیجه برسند که براساس آن ظرفیت تولید صنعت پتروشیمی به ۱۰۰ میلیون تن در سال می‌رسد. [n] وی با اشاره به پیشرفت بسیار مطلوب بعضی از طرح‌های جهش سوم صنعت پتروشیمی، افزود: براساس برنامه‌ریزی انجام شده تا پایان سال ۱۴۰۴، جهش سومی در مقدار تولید محصولات پتروشیمی رخ می‌دهد که با تحقق آن ظرفیت تولید کحصولات پتروشیمی به سالانه ۱۳۳ میلیون تن خواهد رسید. [n] سرپرست مدیریت برنامه‌ریزی و توسعه شرکت ملی صنایع پتروشیمی با اشاره به اینکه تنوع محصولات پتروشیمی پس از بهره‌برداری از طرح‌های جهش دوم و سوم پتروشیمی افزایش قابل ملاحظه‌ای خواهد داشت، ادامه داد: تامین نیاز داخل در اولویت شرکت ملی صنایع پتروشیمی است و پس از تحقق این جهش‌ها می‌توان محصولات وارداتی با مصرف بالا را در داخل تولید کرد. [n] گودرزی‌فراهانی تصریح کرد: شرکت ملی صنایع پتروشیمی افزون بر دریافت خوراک از بالادست و تکمیل زنجیره و جلوگیری از خام‌فروشی، برای متنوع‌ کردن سبد محصولات صنعت پتروشیمی و تکمیل زنجیره ارزش نیز گام برمی دارد. [n] وی با اشاره به اینکه تعداد محصولات اصلی تولیدی از ۹۰ محصول کنونی به ۱۰۵ محصول در پایان سال ۱۴۰۴ می‌رسد، گفت:: با متنوع‌ کردن محصولات سبد پتروشیمی، مواد اولیه مورد نیاز صنایع پایین‌دست پتروشیمی نیز تامین می‌شود و و نیاز داخل به بعضی از محصولات مانند آکریلونیتریل که هم‌اکنون وارد می‌شود، تامین خواهد شد. [n] سرپرست مدیریت برنامه‌ریزی و توسعه شرکت ملی صنایع پتروشیمی با اشاره به برنامه‌های این شرکت برای مجتمع‌های پتروشیمی، گفت: ۲ برنامه برای مجتمع‌های پتروشیمی در دستور کار است؛ نخست تلاش می‌شود تولید مجتمع‌ها به ظرفیت اسمی برسند، دو اینکه از طرح‌های توسعه‌ای و ساخت واحدهای جدید با خوراک تولیدی خود مجتمع‌ها استقبال می‌شود و برای صدور مجوز اجرای آنها سختگیری نخواهد شد."
headline = generate_headline(article_text)
print(headline)
# سبد محصولات پتروشیمی متنوع می‌شود

ظرفیت تولید کحصولات پتروشیمی به ۱۰۰ میلیون تن می رسد


In [19]:
article_text = "به گزارش شانا به نقل از شرکت پالایش گاز شهید هاشمی‌نژاد، جمعی از کارشناسان این پالایشگاه با هدف معرفی محصول کود گوگرد بنتونیتی به کشاورزان در شانزدهمین نمایشگاه تخصصی کشاورزی، دام، طیور، ماشین‌آلات و صنایع وابسته در اردبیل حضور پیدا کردند و به معرفی دستاوردهای این شرکت پرداختند. [n] بر اساس این گزارش، دشت مغان قطب کشاورزی در منطقه اردبیل است و بیش از ۸۵ هزار هکتار زمین کشاورزی دارد که همه آنها نیاز به استفاده از کود گوگرد بنتونیتی دارند، به همین دلیل، کارشناسان پالایشگاه گاز هاشمی‌نژاد در راستای توسعه بازار و معرفی محصولات جانبی پالایشگاه (کود گوگرد بنتونیتی) به بازار داخلی در این نمایشگاه حضور یافتند. [n] شرکت در این نمایشگاه سبب شد تعداد زیادی از بازرگانان ایرانی فعال در حوزه گوگرد ضمن دیدار از غرفه شرکت پالایش گاز شهید هاشمی‌نژاد خواستار خرید محصول کود گوگرد بنتونیتی این مجتمع گازی شوند. [n] شرکت پالایش گاز شهید هاشمی‌نژاد در کنار رسالت اصلی خود مبنی بر تامین گاز مشترکان شمال و شمال شرق‌کشور، محصولات جانبی ارزشمند مانند گوگرد را تولید می‌کند، به گونه‌ای که اکنون با تولید روزانه بیش از ۲ هزار تن گوگرد مذاب، تولید روزانه ۶۰۰ تن گوگرد بنتونیتی (مطابق نیاز بازار) و تولید سالانه بیش از ۲۰ هزار تن گوگرد بنتونیتی، بزرگ‌ترین تولیدکننده گوگرد در کشور است. [n] کود گوگرد بنتونیتی نیز در راستای ارتقای کیفی خاک و تامین عناصر مورد نیاز رشد گیاهان تاثیری بسزا در بازده تولید محصولات کشاورزی دارد. [n] کود گوگرد بنتونیتی تولیدی پالایشگاه هاشمی‌نژاد، با استانداردهای جهانی تولید شده و جایگاه خود را در بازارهای داخلی و خارجی کسب کرده است. مطابق برنامه‌ریزی‌های انجام شده و بررسی روند رو به رشد مصرف این کود در بازار، پیش‌بینی می‌شود میزان مصرف کود گوگرد بنتونیتی تا پنج سال آینده به ۱۰۰ هزار تن در سال برسد."
headline = generate_headline(article_text)
print(headline)
# معرفی گوگرد بنتونیتی پالایشگاه خانگیران در نمایشگاه کشاورزی اردبیل

معرفی محصولات جانبی پالایشگاه هاشمی نژاد به کشاورزان


In [23]:
article_text = "به گزارش خبرنگار ایمنا، سعید نظری در صفحه اینستاگرام خود نوشت: «در حالیکه شهر شیراز نخستین روزهای سال ۹۹ را با شیوع بیماری کرونا با حساسیت ویژه طی می‌کند، اعضای شورای اسلامی شهر شیراز در تلاشند همسو با ستاد ملی مبارزه با کرونا، مخاطرات ناشی از شیوع این ویروس منحوس را برای شهروندان کاهش دهند و از طرفی با برنامه‌ریزی جدید و فوق العاده، ساختار مدیریت شهری از حرکت و کوشش باز نایستد. [n] بر اساس شرایط موجود کلان‌شهر شیراز، تامین منابع مالی پروژه‌های عمرانی و جاری، انجام فعالیت‌های عمرانی با تاکید بر شعار انسان محور، رفتار ترافیکی شهروندان، تامین نیازهای ضروری شهروندان و ضدعفونی و گندزدایی شهر شیراز از جمله دغدغه‌های مهم این روزهای شورای اسلامی شهر شیراز است که توسط اعضای محترم در حال پیگیری است. [n] شورای اسلامی شهر شیراز اقدامات مدیریت کلانشهر شیراز برای مقابله با شیوع کرونا را مثبت و موفق ارزیابی کرده و انتظار می‌رود با برنامه‌ریزی‌های در حال انجام، روند عمرانی و شهرسازی در کلان‌شهر شیراز با فروکش کردن حساسیت‌های کرونایی به روند عادی خود باز گردد و با تلاش مضاعف در پایان دوره پنجم، رضایت شهروندان عزیز بیش از پیش جلب شود.»"
headline = generate_headline(article_text)
print(headline)
# روند عمرانی شیراز با فروکش کردن کرونا عادی می‌شود


اقدامات مدیریت کلانشهر شیراز برای مقابله با کرونا مثبت ارزیابی کرد


In [24]:
article_text = " به گزارش شانا، سیدباقر مرتضوی، مشاور وزیر نفت و مدیرکل اچ اس یی و پدافند غیرعامل وزارت نفت از فرخ عمویی یکی از کارکنان حراست به منظور اطفای آتش سوزی کمپ مسکونی شیرینو شرکت مجتمع گاز پارس جنوبی قدردانی کرد. [n] کمپ شیرینو در ۲۳ اردیبهشت ماه ۹۶ دچار حادثه آتش سوزی شد که با اقدام فوری و سنجیده فرخ عمویی و همکاران در همان لحظات اولیه اطفا و از گسترش حادثه و پیامدهای آن جلوگیری شد. [n] در لوح سپاس مشاور وزیر نفت و مدیرکل اچ اس یی و پدافند غیرعامل وزارت نفت به فرخ عمویی آمده است: [n] بدون شک کارکنان زحمتکش حراست که به صورت شبانه روزی در تاسیسات عملیاتی و ساختمان‌ها حضور داشته و به کمک ابزارهای نظارتی کنترل وضعیت محل را در اختیار دارند، می‌توانند نقش موثر و چشمگیری را در لحظات اولیه وقوع رخداد جهت کنترل حوادث به خصوص حریق‌ها ایفا کنند که این مهم از سوی وزیر محترم نفت مورد تاکید قرار گرفته و در پرتو رهنمودهای ایشان در قالب تفاهم نامه‌ای میان سازمان حراست و اداره کل اچ‌اس‌یی و پدافند غیرعامل وزارت نفت چهره عملیاتی یافته است. [n] از جمله مصادیق این امر، حضور فوری و انجام اقدامات سنجیده و موثر جنابعالی و همکاران در نخستین لحظات آتش سوزی برای اطفای حریق اقامتگاه کارکنان مجتمع گازی پارس جنوبی است که منجر به مهار آتش سوزی شده و از بروز صدمات و خسارت بیشتر جلوگیری کرد. به پاس قدردانی از اقدام‌های ارزشمند جنابعالی و همکاران، این لوح تقدیر تقدیم می‌شود. امید است کلیه همکاران محترم حراست با بهره گیری از ارشادات وزیر محترم نفت نقش موثر و چشمگیر خود را در نخستین لحظات رخداد به منظور مدیریت شرایط اضطراری ایفا کنند. [n] با توجه به تاکید مقام عالی وزارت نفت بر عملکرد صحیح و فوری در لحظات اولیه وقوع رویداد در تاسیسات صنعت نفت به منظور پیشگیری و کنترل پیامدهای حوادث، در سی و پنجمین جلسه شورای معاونان وزارت نفت در ۲۷ بهمن ۹۳، مقرر شد از ظرفیت و توانمندی کارکنان حراست به منظور کمک به واحدهای مسئول ایمنی و مدیریت شرایط اضطراری به نحو مناسبی بهره گرفته شود. [n] بنابراین تفاهم‌نامه همکاری مشترک میان اداره کل اچ‌اس‌یی و پدافند غیرعامل و سازمان حراست صنعت نفت در ۲۷ دی‌ماه ۹۴ امضا و ساز و کارهای اجرایی شدن رهنمودهای وزیرمحترم و مصوبه شورای معاونان از جمله آموزش‌های لازم پیش بینی شد که با تلاش و اهتمام مدیریت اچ اس یی، ادارات آموزش و حراست شرکت‌های اصلی و تابعه بخش قابل توجه کارکنان حراست آموزش‌های لازم را طی کرده‌اند."
headline = generate_headline(article_text)
print(headline)
#  قدردانی از اقدام ایثارگرانه نیروی حراست در اطفای حریق کمپ مسکونی شیرینو

فرخ عمویی یکی از کارکنان حراست قدردانی کرد


In [25]:
article_text = "رئیس بهره برداری پالایشگاه سوم مجتمع پارس جنوبی امروز، دوشنبه (۲۳ شهریورماه) در گفتگو با خبرنگار شانا اظهار کرد: این پالایشگاه برنامه تعمیرات اساسی خود را براساس حفظ ضریب دسترس پذیری ۹۷ درصدی سال گذشته ابتدا در بازه ۱۷ روزه برنامه ریزی کرده بود. [n] محسن عطارزاده ادامه داد: پالایشگاه سوم با بررسی دقیق شرایط عملیاتی در طول سال، کسب اطمینان از تولید پایدار گاز و قبول ریسک انجام فعالیتهای ضروری پیش از زمستان و همچنین با مد نظر قرار گرفتن حذف تعمیرات بالادستی (سکوهای پالایشگاه)، تصمیم به اجرا نکردن پروژه تعمیرات اساسی سال جاری گرفت تا در اقدامی آگاهانه و رو به جلو، رکورد تولید پایدار ۳۶۵ روزه خود را در سال جاری ثبت کند. [n] وی افزود: به این ترتیب پالایشگاه سوم پارس جنوبی در کنار حفظ تولید سالهای ۹۲ و ۹۳ که ضریب اطمینان ۹۹٫۹۷ درصدی را ارائه کرده است، در سال جاری علاوه بر ضریب اطمینان تولید، ضریب دسترس پذیری بالاتری را تثبیت می‌کند. [n] به گفته رئیس بهره برداری پالایشگاه سوم مجتمع پارس جنوبی، با حذف تعمیرات اساسی و تداوم ۱۷ روزه برداشت گاز از مخازن مشترک، تولید پالایشگاه نسبت به سال گذشته سه درصد افزایش خواهد داشت که درآمدی بیش از ۴۰۰ میلیارد تومان، معادل یک درصد بودجه عمرانی کل کشور را در بر می‌گیرد. [n] عطارزاده از اعتماد مدیرعامل مجتمع، مدیر پالایشگاه، اهتمام همکاران واحدهای بهره ‌برداری، مهندسی، تعمیرات، بازرسی فنی و سایر واحدهای پشتیبانی در دستیابی به این موفقیت، تقدیر و تشکر کرد."
headline = generate_headline(article_text)
print(headline)
#   درآمد پالایشگاه سوم پارس جنوبی ۴۰۰ میلیارد تومان افزایش می‌یابد

رکورد تولید پایدار پالایشگاه سوم پارس جنوبی ثبت می شود


In [ ]:
token = "your hugging face token"

trainer.push_to_hub("APredator/mt5-persian-title", token = token)